# GenHack 4 - ChefAI - Metrics & Quantitative Insight

- **Group**: ChefAI
- **Period**: 3 - Metrics & Quantitative Insight


## Introduction


In [13]:
import glob
import warnings
import math
import typing
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
import seaborn as sns

from scipy import stats
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import Normalize

In [14]:
# ECA&D station functions
def dms_to_decimal(dms_series) -> pd.Series:
    """DMS (degrees, minutes, seconds) to decimal degrees conversion."""
    # Extract sign
    signs = np.where(dms_series.str[0] == "+", 1, -1)
    # Remove sign and split by ':'
    parts = dms_series.str[1:].str.split(":", expand=True).astype(float)
    # Calculate decimal degrees
    decimal = signs * (parts[0] + parts[1] / 60 + parts[2] / 3600)
    return decimal


def eca_read_stations(path: str) -> gpd.GeoDataFrame:
    """Load ECA&D stations from a CSV file and convert to GeoDataFrame."""

    stations_df = pd.read_csv(path, skiprows=17, skipinitialspace=True)

    # Strip whitespace from column names
    stations_df.columns = stations_df.columns.str.strip()

    # Strip whitespace from CN and STANAME columns
    stations_df["CN"] = stations_df["CN"].str.strip()
    stations_df["STANAME"] = stations_df["STANAME"].str.strip()

    # Convert degrees-minutes-seconds to decimal degrees
    lat_decimal = dms_to_decimal(stations_df["LAT"])
    lon_decimal = dms_to_decimal(stations_df["LON"])

    # Create GeoDataFrame
    geometries = gpd.points_from_xy(lon_decimal, lat_decimal)
    stations_gdf = gpd.GeoDataFrame(stations_df, geometry=geometries, crs="EPSG:4326")

    # Drop original LAT and LON columns
    stations_gdf.drop(columns=["LAT", "LON"], inplace=True)

    return stations_gdf


def eca_read_station_tx(path: str) -> pd.DataFrame:
    """Read TX data for a single ECA&D station."""

    station_df = pd.read_csv(path, skiprows=20, skipinitialspace=True, engine="c")

    # Convert DATE column to datetime
    station_df["DATE"] = pd.to_datetime(station_df["DATE"], format="%Y%m%d")
    # Convert from tenths of degree C to degree C
    station_df["TX"] = station_df["TX"] / 10.0
    # Where Q_TX is 9, set TX to NaN
    station_df = station_df[station_df["Q_TX"] == 0]
    station_df.drop(columns=["Q_TX"], inplace=True)

    return station_df


def eca_read_all_stations_tx(paths: list[str]) -> pd.DataFrame:
    """Read TX data for multiple ECA&D stations and concatenate into a single DataFrame."""
    all_stations_df = pd.DataFrame()

    for path in tqdm(paths, desc="Reading station TX data"):
        station_df = eca_read_station_tx(path)
        all_stations_df = pd.concat([all_stations_df, station_df], ignore_index=True)

    return all_stations_df


In [15]:
# ERA5 loader
def era5_read_xr(path_glob: str) -> xr.Dataset:
    """Load ERA5 dataset from one or more NetCDF files."""
    paths = list(glob.glob(path_glob))

    dataset = xr.open_mfdataset(paths, combine="by_coords")
    dataset.rio.write_crs("EPSG:4326", inplace=True)

    return dataset

In [16]:
# NDVI functions
def ndvi_read_xr(path: str) -> xr.DataArray:
    """Load NDVI raster lazily and rescale from 0–255 to -1..1."""
    # lazy read
    ndvi = rioxr.open_rasterio(path, masked=True, chunks=True, lock=False)
    ndvi = typing.cast(xr.DataArray, ndvi)
    ndvi = ndvi.squeeze()

    # mask nodata (255)
    ndvi = ndvi.where(ndvi != 255)

    # lazy scaling (dask)
    ndvi = ndvi * (2 / 255) - 1

    return ndvi


def ndvi_read_xr_bbox_clip(path: str, gdf: gpd.GeoDataFrame) -> xr.DataArray:
    """
    Load NDVI lazily and clip to the bbox of the GeoDataFrame.
    CRS is handled correctly by reprojecting the region to raster CRS.
    """
    # Load NDVI
    ndvi = ndvi_read_xr(path)
    # Convert gdf to raster CRS
    gdf_r = gdf.to_crs(ndvi.rio.crs)
    minx, miny, maxx, maxy = gdf_r.total_bounds
    # Clip to bbox
    ndvi_filtered = ndvi.rio.clip_box(minx=minx, miny=miny, maxx=maxx, maxy=maxy)
    return ndvi_filtered
